In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
def dfdx(x,f):
    
    # dz/dx = -y
    # y = f[0]
    # dy/dx = z
    # z = f[1]
    
    y = f[0]
    z = f[1]
    
    # returning the derivatives
    dydx = np.zeros_like(f)
    dydx[0] = z
    dydx[1] = -1*y
    
    return dydx

In [ ]:
def cash_karp_core_mv(x_i,y_i,nv,h,f):
    
    ni = 7
    nj = 6
    ci = np.zeros(ni)
    aij = np.zeros( (ni,nj) )
    bi = np.zeros(ni)
    bis = np.zeros(ni)
    
    # inputing the values
    ci[2] = 1./5.
    ci[3] = 3./10.
    ci[4] = 3./5.
    ci[5] = 1.
    ci[6] = 7./8.
    
    
    # j = 1
    aij[2,1] = 1./5
    aij[3,1] = 3./40.
    aij[4,1] = 3./10.
    aij[5,1] = -11./54.
    aij[6,1] = 1631./55296.
    
    # j = 2
    aij[3,2] = 9./40.
    aij[4,2] = -9./10.
    aij[5,2] = 5./2.
    aij[6,2] = 175./512.
    
    # j = 3
    aij[4,3] = 6./5.
    aij[5,3] = -70./27.
    aij[6,3] = 575./13824.
    
    # j = 4
    aij[5,4] = 35./27.
    aij[6,4] = 44275./110592.
    
    # j = 5
    aij[6,5] = 253./4096.
    
    #bi
    bi[1] = 37./378.
    bi[2] = 0.
    bi[3] = 250./621.
    bi[4] = 125./594.
    bi[5] = 0.0
    bi[6] = 512./1771.
    
    #bis
    bis[1] = 2825./27648.
    bis[2] = 0.0
    bis[3] = 18575./48384.
    bis[4] = 13525./55296.
    bis[5] = 277./14336.
    bis[6] = 1./4.
    
    # k array
    ki = np.zeros((ni,nv))
    
    # computing ki
    for i in range(1,ni):
        # xn+1 for i
        xn = x_i + ci[i]*h
        
        # computing temp y
        yn = y_i.copy()
        for j in range(1,i):
            yn += aij[i,j]*ki[j,:]
            
        # K value
        ki[i,:] = h*f(xn,yn)
        
        
    ynpo = y_i.copy()
    ynpos = y_i.copy()
        
    for i in range(1,ni):
        ynpo += bi[i] *ki[i,:]
        ynpos += bis[i]*ki[i,:]
            
    # get error
    Delta = np.fabs(ynpo-ynpos)
        
        
    return ynpo,Delta

In [ ]:
def cash_karp_mv_ad(dfdx,x_i,y_i,nv,h,tol):
    
    # Saftey scale
    SAFETY = 0.9
    H_NEW_FAC = 2.0
    
    # maximum number of iterations
    imax = 1000
    
    # iteration variable
    i = 0
    
    # Error
    Delta = np.full(nv,2*tol)
    
    # the step
    h_step = h
    
    # adjusting step
    while(Delta.max()/tol>1.0):
        
        # new y and estimate error
        y_ipo, Delta = cash_karp_core_mv(x_i,y_i,nv,h_step,dfdx)
        
        # take smaller step if error is large
        if(Delta.max()/tol>1.0):
            
            # decrease step
            h_step *= SAFETY * (Delta.max()/tol)**(-0.25)
            
        # checking iteration
        if (i>=imax):
            print("Too many iterations in cash_karp_mv_ad()")
            raise StopIteration("Ending after i = ",1)
            
        i += 1
        
    h_new = np.fmin(h_step * (Delta.max()/tol)**(-0.9), h_step * H_NEW_FAC)
    
    # return answer
    return y_ipo, h_new, h_step

In [ ]:
def cash_karp_mv(dfdx,a,b,y_a,tol,verbose=False):
    
    # a is lower bound
    # b is upper bound
    # y_a boundry condition
    # and tolerance is tol
    
    # starting step
    xi = a
    yi = y_a.copy()
    
    # initial starting step
    h = 1.0e-4 * (b-a)
    
    imax = 1000
    
    # iteration variable
    i = 0
    
    nv = len(y_a)
    
    #iniital conditions
    x = np.full(1,a)
    y = np.full( (1,nv), y_a)
    
    flag = True
    
    while(flag):
        
        y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
        
        # updating the step
        h = h_new
        
        if(xi+h_step>b):
            # limit step at b
            h = b - xi
            
            y_ipo, h_new, h_step = cash_karp_mv_ad(dfdx,xi,yi,nv,h,tol)
            
            flag = False
            
        xi += h_step
        yi = y_ipo.copy()
        
        # adding the step
        x = np.append(x,xi)
        y_ipo = np.zeros((len(x),nv))
        y_ipo[0:len(x)-1,:] = y[:]
        y_ipo[-1,:] = yi[:]
        del y
        y = y_ipo
        
        
        if(i>=max):
            print("Maximum iterations reached.")
            raise StopIteration("Iteration number = ",i)
            
        i +=1
        
        # Output information
        if(verbose):
            s = "i = %3d/tx = %9.8f/ty = %9.8f/th = %9.8f/tb = %9.8f" % (i, xi,yi[0],h_step,b)
        
        if(xi==b):
            flag = False
        
    return x, y       

In [ ]:
a = 0.0
b = 2.0*np.pi
y_0 = np.zeros(2)
y_0[0] = 0.0
y_0[1] = 1.0
nv = 2

tolerance = 1.0e-6
x, y = cash_karp_mv(dfdx,a,b,y_0,tolerance,verbose=True)
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='z(x)')
plt.plot(x,np.sin(x),label='sin(x)')
plt.plot(x,np.cos(x),label='cos(x)')
plt.xlim([0,2*np.pi])
plt.ylim([-1.2,1.2])
plt.legend(frameon=False)

In [ ]:
plt.plot(x, y[:,0]=np.sin(x), label="y(x) Error")
plt.plot(x, y[:,1]=np.cos(x), label="z(x) Error")
plt.legend()